In [10]:
# Import required libraries
import os
import logging
import pandas as pd
import numpy as np
from sklearn import metrics
from sktime.transformations.panel.rocket import Rocket
from sklearn.linear_model import RidgeClassifierCV
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [3]:
# Configure logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

In [11]:
# Define Rocket Transformer Classifier
class RocketTransformerClassifier:
    def __init__(self):
        self.classifiers_mapping = {}

    def fit_rocket(self, x_train, y_train, kernels=10000):
        # Initialize and fit Rocket transformer
        rocket = Rocket(num_kernels=kernels, normalise=False)
        rocket.fit(x_train)
        x_training_transform = rocket.transform(x_train)

        # Normalize the transformed data
        scaler = StandardScaler()
        x_training_transform = scaler.fit_transform(x_training_transform)

        # Train RidgeClassifier with normalized transformed data
        classifier = RidgeClassifierCV(alphas=np.logspace(-3, 3, 10))
        classifier.fit(x_training_transform, y_train)

        # Store the transformer, scaler, and classifier
        self.classifiers_mapping["transformer"] = rocket
        self.classifiers_mapping["scaler"] = scaler
        self.classifiers_mapping["classifier"] = classifier

    def predict_rocket(self, x_test, y_test):
        # Retrieve transformer, scaler, and classifier
        rocket = self.classifiers_mapping["transformer"]
        scaler = self.classifiers_mapping["scaler"]
        classifier = self.classifiers_mapping["classifier"]
    
        # Transform and normalize test data
        x_test_transform = rocket.transform(x_test)
        x_test_transform = scaler.transform(x_test_transform)
    
        # Predict and evaluate
        predictions = classifier.predict(x_test_transform)
        accuracy = metrics.accuracy_score(y_test, predictions)
        confusion_matrix = metrics.confusion_matrix(y_test, predictions)
        classification_report = metrics.classification_report(y_test, predictions)
    
        logger.info("-----------------------------------------------")
        logger.info(f"Accuracy: {accuracy}")
        logger.info("\nConfusion Matrix:\n" + str(confusion_matrix))
        logger.info("\nClassification Report:\n" + classification_report)
    
        return accuracy, confusion_matrix, classification_report

# Define function to load time-series data
def load_time_series_data(input_dir):
    """
    Load all time-series CSV files in the input directory and prepare
    the data for Rocket classifier.
    """
    x_data = []
    y_data = []

    # Iterate through each CSV file
    for file_name in os.listdir(input_dir):
        if file_name.endswith('.csv'):
            file_path = os.path.join(input_dir, file_name)

            # Load CSV file
            df = pd.read_csv(file_path)

            # Extract label and time-series data
            label = df.iloc[0]['class']  # 'class' 열에서 라벨을 가져옵니다.
            # label = df['class'].iloc[0]  # Assuming 'class' column exists in all files
            time_series = df.iloc[:, 3:].values  # Exclude non-time-series columns
            
            x_data.append(time_series)
            y_data.append(label)

    return np.array(x_data), np.array(y_data)


In [12]:
# Load the data
input_dir = '/root/juno/time_series_data'  # 데이터 디렉토리 경로
x_data, y_data = load_time_series_data(input_dir)

# 데이터 확인
print(f"x_data shape: {x_data.shape}")
print(f"y_data shape: {y_data.shape}")
print(f"Sample label: {y_data[0]}")
print(f"Sample time-series data shape: {x_data[0].shape}")

x_data shape: (348, 32, 132)
y_data shape: (348,)
Sample label: 377
Sample time-series data shape: (32, 132)


In [13]:
# Split the data into train and test sets
logger.info("Splitting data into train/test sets...")
x_train, x_test, y_train, y_test = train_test_split(x_data, y_data, test_size=0.2, random_state=42, stratify=y_data)

INFO:__main__:Splitting data into train/test sets...


In [14]:
# Initialize Rocket classifier
logger.info("Initializing Rocket classifier...")
rocket_classifier = RocketTransformerClassifier()

INFO:__main__:Initializing Rocket classifier...


In [15]:
# Train the classifier
logger.info("Training Rocket classifier...")
rocket_classifier.fit_rocket(x_train, y_train)

INFO:__main__:Training Rocket classifier...


In [16]:
# Test the classifier
logger.info("Testing Rocket classifier...")
accuracy, confusion_matrix, classification_report = rocket_classifier.predict_rocket(x_test, y_test)

INFO:__main__:Testing Rocket classifier...
/root/miniconda3/envs/condatest/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/root/miniconda3/envs/condatest/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/root/miniconda3/envs/condatest/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average

In [20]:
# Save the results to a file
output_dir = '/root/juno'

results_file = os.path.join(output_dir, 'classification_results.txt')
with open(results_file, 'w') as f:
    f.write("Accuracy: {}\n".format(accuracy))
    f.write("\nConfusion Matrix:\n")
    f.write(str(confusion_matrix))
    f.write("\nClassification Report:\n")
    f.write(classification_report)

logger.info("Results saved to: " + results_file)


INFO:__main__:Results saved to: /root/juno/classification_results.txt
